In [3]:
!pip install opencv-python

  Using cached opencv_python-4.7.0.72-cp37-abi3-win_amd64.whl (38.2 MB)


In [4]:
import pandas as pd
import cv2
import os
from PIL import Image
from sklearn.model_selection import train_test_split
import numpy as np

In [5]:
img_dir = '../cropped_img/II/'

img_size = 640, 320

img_ds = []
label = []

for file in os.listdir(img_dir):
    img = cv2.imread(img_dir+file)
    H, W = img.shape[:2]
    rs_img = cv2.resize(img,img_size)
    label.append(file.split('_')[-1].split('.')[0])
    img_ds.append(rs_img)

    

In [6]:
len(img_ds)

633

In [7]:
len(label)

633

In [26]:
y = np.array([1 if x == 'fa' else 0 for x in label])

In [27]:
img_ds = np.array(img_ds)

In [28]:
img_ds.shape

(633, 320, 640, 3)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(img_ds, y, test_size=0.2, stratify=label, random_state=42)

# Further split the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [30]:
X_train.shape

(404, 320, 640, 3)

In [31]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the model architecture
model = Sequential()

# First convolutional layer with 32 filters, a 3x3 kernel, and ReLU activation
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(320, 640, 3)))

# Max pooling layer with a 2x2 pool size
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolutional layer with 64 filters, a 3x3 kernel, and ReLU activation
model.add(Conv2D(64, (3, 3), activation='relu'))

# Max pooling layer with a 2x2 pool size
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Fully connected layer with 512 units and ReLU activation
model.add(Dense(512, activation='relu'))

# Dropout layer with a rate of 0.5 to reduce overfitting
model.add(Dropout(0.5))

# Output layer with a sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

In [32]:
# Compile the model with binary cross-entropy loss, Adam optimizer, and accuracy metric
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model with the specified batch size and number of epochs
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
13/13 [==============================] - 62s 4s/step - loss: 2993.9246 - accuracy: 0.6807 - val_loss: 2.7516 - val_accuracy: 0.6863
Epoch 2/10
13/13 [==============================] - 47s 4s/step - loss: 1.4785 - accuracy: 0.7079 - val_loss: 0.6140 - val_accuracy: 0.6863
Epoch 3/10
13/13 [==============================] - 48s 4s/step - loss: 0.4936 - accuracy: 0.8193 - val_loss: 0.6302 - val_accuracy: 0.6863
Epoch 4/10
13/13 [==============================] - 47s 4s/step - loss: 0.4564 - accuracy: 0.8218 - val_loss: 0.7254 - val_accuracy: 0.6863
Epoch 5/10
13/13 [==============================] - 47s 4s/step - loss: 0.4674 - accuracy: 0.8218 - val_loss: 0.6234 - val_accuracy: 0.6863
Epoch 6/10
13/13 [==============================] - 46s 4s/step - loss: 0.4729 - accuracy: 0.8168 - val_loss: 0.6636 - val_accuracy: 0.6863
Epoch 7/10
13/13 [==============================] - 49s 4s/step - loss: 0.4591 - accuracy: 0.8168 - val_loss: 0.6638 - val_accuracy: 0.6863
Epoch 8/10
13/13 

In [43]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

4/4 - 4s - loss: 0.4974 - accuracy: 0.8031 - 4s/epoch - 915ms/step


In [45]:
sum(y_test),len(y_test)

(100, 127)